In [1]:
import os

from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Langchain configuration
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
# Project configuration
os.environ["LANGCHAIN_PROJECT"] = "LangGraph"

# OpenAI API Key
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# Human-in-the-loop


In [33]:
from typing import Annotated
from typing_extensions import TypedDict

from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

# Create memory
memory = SqliteSaver.from_conn_string(":memory:")


# Create a state
class State(TypedDict):
    messages: Annotated[list, add_messages]


# Create a graph with the state
graph_builder = StateGraph(State)

# Create a tool
tool = TavilySearchResults(max_results=2)
tools = [tool]

# Create a llm and bind tools
llm = ChatOpenAI()
llm_with_tools = llm.bind_tools(tools)


# Create a node - chatbot
def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}


# Add chatbot node to the graph
graph_builder.add_node("chatbot", chatbot)

# Create a tool node from tools and add it to the graph
tool_node = ToolNode(tools=[tool])
graph_builder.add_node("tools", tool_node)

# Add conditional edges to the graph
graph_builder.add_conditional_edges(
    "chatbot",
    tools_condition,
)
graph_builder.add_edge("tools", "chatbot")
graph_builder.set_entry_point("chatbot")

Add `interrupt_before` when compiling the graph.


In [34]:
graph = graph_builder.compile(
    checkpointer=memory,
    # This is new!
    interrupt_before=["tools"],
    # Note: can also interrupt __after__ actions, if desired.
    # interrupt_after=["tools"]
)

Let's try to invoke the graph


In [35]:
user_input = "I'm learning LangGraph. Could you do some research on it for me?"
config = {"configurable": {"thread_id": "1"}}
# The config is the **second positional argument** to stream() or invoke()!
events = graph.stream(
    {"messages": [("user", user_input)]}, config, stream_mode="values"
)
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================ Human Message =================================

I'm learning LangGraph. Could you do some research on it for me?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (call_cZGXTMhf0okoGMt03dNecP47)
 Call ID: call_cZGXTMhf0okoGMt03dNecP47
  Args:
    query: LangGraph


Read the state


In [36]:
snapshot = graph.get_state(config)
snapshot.next

('tools',)

Print the messages within the snapshot


In [37]:
existing_message = snapshot.values["messages"][-1]
existing_message.tool_calls

[{'name': 'tavily_search_results_json',
  'args': {'query': 'LangGraph'},
  'id': 'call_cZGXTMhf0okoGMt03dNecP47'}]

In [38]:
# `None` will append nothing new to the current state, letting it resume as if it had never been interrupted
events = graph.stream(None, config, stream_mode="values")
for event in events:
    if "messages" in event:
        event["messages"][-1].pretty_print()

================================= Tool Message =================================
Name: tavily_search_results_json

[{"url": "https://langchain-ai.github.io/langgraph/", "content": "LangGraph is a library for building stateful, multi-actor applications with LLMs, used to create agent and multi-agent workflows. Compared to other LLM frameworks, it offers these core benefits: cycles, controllability, and persistence. LangGraph allows you to define flows that involve cycles, essential for most agentic architectures ..."}, {"url": "https://github.com/langchain-ai/langgraph", "content": "LangGraph is a library for building stateful, multi-actor applications with LLMs using regular python functions or JS. Learn how to create graphs with nodes, edges, state, tools, and conditional edges for agentic behaviors."}]
================================== Ai Message ==================================

LangGraph is a library for building stateful, multi-actor applications with LLMs (Large Language Model